Context Managers and the With Statement
=======================================

There are many situations where you find yourself writing code which follows a pattern like:

    # create/aquire some resource
    ...
    try:
        # do something with the resource
        ...
    finally:
        # destroy/release the resource
        ...

You see this sort of pattern in opening and closing files, acquiring locks when using threads, in network programming, and when using databases, amongst many other situations.

Python introduced the `with` statement to handle exactly this sort of situation, and we have already seen this in the discussion of file-IO where:

In [ ]:
with open('my_file', 'w') as fp:
    # do stuff with fp
    data = fp.write("Hello world")

is the same as:

In [ ]:
fp = open('my_file', 'w')
try:
    # do stuff with f
    data = fp.write("Hello world")
finally:
    fp.close()

but much clearer.

So how can we create objects which use this pattern in our own code?

How the With Statement Works
----------------------------

The with statement, in its simplest form, looks like this:

    with <expression>:
        <block>

When Python executes this code, it evaluates the expression, which should return an object which implements the "context manager" protocol, that is an object with two special methods: `__enter__` and `__exit__`.

Looking at our file object:

In [ ]:
print fp.__enter__
print fp.__exit__

The `__enter__` method gets called before the block is run, and the `__exit__` method gets called after the block completes, even if there is an exception in the code.

A minimal context manager class might look like this:

In [ ]:
class ContextManager(object):
    
    def __enter__(self):
        print "Entering"
    
    def __exit__(self, exc_type, exc_value, traceback):
        print "Exiting"

You can use this as follows:

In [ ]:
with ContextManager():
    print "  Inside the with statement"

The `__exit__` method gets run, even if there is an exception.

In [ ]:
with ContextManager():
    print 1/0

You have the option of returning a value from the `__enter__()` method.  This is the value that is given to the `as` clause of a `with` statement:

In [ ]:
class ContextManager(object):
    
    def __enter__(self):
        print "Entering"
        return "my value"
    
    def __exit__(self, exc_type, exc_value, traceback):
        print "Exiting"

In [ ]:
with ContextManager() as value:
    print value

A very common pattern is for the `__enter__()` method to return the context manager itself.  The file object does this, for example:

In [ ]:
fp = open("my_file", "r")
print fp.__enter__()

Getting your code to do this is straightforward:

In [ ]:
class ContextManager(object):
    
    def __enter__(self):
        print "Entering"
        return self
    
    def __exit__(self, exc_type, exc_value, traceback):
        print "Exiting"

In [ ]:
with ContextManager() as value:
    print value

Error Handling in Context Managers
----------------------------------

The `__exit__()` method gets passed information about any exceptions which happened in the block of the with statement.  If no exception occurred, then the values will be none, otherwise they are the exception type, the exception object, and the traceback associated with the exception.

In [ ]:
class ContextManager(object):
    
    def __enter__(self):
        print "Entering"
    
    def __exit__(self, exc_type, exc_value, traceback):
        print "Exiting"
        if exc_type is not None:
            print "  Exception:", exc_value

In [ ]:
with ContextManager():
    print 1/0

Note that in this case, we let the error continue.  If you want to suppress the exception, you can return `True` from the `__exit__()` method:

In [ ]:
class ContextManager(object):
    
    def __enter__(self):
        print "Entering"
    
    def __exit__(self, exc_type, exc_value, traceback):
        print "Exiting"
        if exc_type is not None:
            print "  Exception suppresed:", exc_value
            return True

In [ ]:
with ContextManager():
    print 1/0

As a concrete and practical example of this sort of error handling, the following class uses Python's database API to safely encapsulate a database transaction.  If our code executes normally, we want to ensure that we `commit()` the transaction, otherwise we want to `rollback()` the transaction.

In [ ]:
class Transaction(object):
    
    def __init__(self, connection):
        self.connection = connection
    
    def __enter__(self):
        return self.connection.cursor()
    
    def __exit__(self, exc_type, exc_value, traceback):
        if exc_value is None:
            # transaction was OK, so commit
            self.connection.commit()
        else:
            # transaction had a problem, so rollback
            self.connection.rollback()

We can then use the code like this:

In [ ]:
import sqlite3 as db
connection = db.connect(":memory:")

with Transaction(connection) as cursor:
    cursor.execute("""CREATE TABLE IF NOT EXISTS addresses (
        address_id INTEGER PRIMARY KEY,
        street_address TEXT,
        city TEXT,
        state TEXT,
        country TEXT,
        postal_code TEXT
    )""")

In [ ]:
with Transaction(connection) as cursor:
    cursor.executemany("""INSERT OR REPLACE INTO addresses VALUES (?, ?, ?, ?, ?, ?)""", [
        (0, '515 Congress Ave', 'Austin', 'Texas', 'USA', '78701'),
        (1, '245 Park Avenue', 'New York', 'New York', 'USA', '10167'),
        (2, '21 J.J. Thompson Ave.', 'Cambridge', None, 'UK', 'CB3 0FA'),
        (3, 'Supreme Business Park', 'Hiranandani Gardens, Powai, Mumbai', 'Maharashtra', 'India', '400076'),
    ])

But if there is a problem:

In [ ]:
with Transaction(connection) as cursor:
    cursor.execute("""INSERT OR REPLACE INTO addresses VALUES (?, ?, ?, ?, ?, ?)""",
        (4, '2100 Pennsylvania Ave', 'Washington', 'DC', 'USA', '78701'),
    )
    raise Exception("out of addresses")

Then the transaction rolls back correctly:

In [ ]:
cursor.execute("SELECT * FROM addresses")
for row in cursor:
    print row

In fact, this pattern is so useful that many Python database libraries implement some variation of it, even though it is not required by the database API.

The `contextlib` Module
-----------------------

Once you have written a few context managers, you may start to notice that you are repeating the same patterns.  Some of these patterns have been gathered into the standard library `contextlib` module.

The simplest of these is the `closing()` function which creates a context manager that guarantees that an object's `close()` method is called, assuming it has one:

In [ ]:
from contextlib import closing
import urllib

with closing(urllib.urlopen('http://www.google.com')) as url:
    html = url.read()

print html[:100]

Another useful tool is the `contextmanager` decorator.  This allows you to decorate a simple generator and create a context manager:

In [ ]:
from contextlib import contextmanager

@contextmanager
def my_contextmanager():
    print "Enter"
    yield
    print "Exit"

In [ ]:
with my_contextmanager():
    print "  Inside the with statement"

(To fully understand this, you might want to review the lectures on generators and decorators.)

If you `yield` a value from the generator, that is given to the `as` clause of the `with` statement:

In [ ]:
@contextmanager
def my_contextmanager():
    print "Enter"
    yield "my value"
    print "Exit"

In [ ]:
with my_contextmanager() as value:
    print value

Error handling is achieved via a `try ... except ...` construct around the `yield`:

In [ ]:
@contextmanager
def my_contextmanager():
    print "Enter"
    try:
        yield
    except Exception as exc:
        print "   Error:", exc
    finally:
        print "Exit"

In [ ]:
with my_contextmanager():
    print 1/0

So we could re-write the transaction example using the `contextmanager` decorator as follows:

In [ ]:
@contextmanager
def transaction(connection):
    cursor = connection.cursor()
    try:
        yield cursor
    except:
        connection.rollback()
        raise
    else:
        connection.commit()

The result is very elegant, easy-to-write code, if perhaps a little magical.

Copyright 2008-2016, Enthought, Inc.<br>Use only permitted under license.  Copying, sharing, redistributing or other unauthorized use strictly prohibited.<br>http://www.enthought.com